In [1]:
!pip3 install -r requirements.txt
!pip3 install ipywidgets


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import logging
import time

import moonrakerpy as moonpy
import cv2
import numpy as np

from camera import CameraWrapper

logging.basicConfig(format='%(asctime)s - %(name)-24s - %(levelname)-7s - %(message)s (%(filename)s:%(lineno)d)')
logger = logging.getLogger('machine')
logger.setLevel(logging.DEBUG)

# Params
stabilize_delay = 3
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

# Passed Objects
printer = moonpy.MoonrakerPrinter('http://localhost')
camera = CameraWrapper()

{'logitech_led1_mode': 'off'}


cheese: no process found


In [3]:
# Run Homing

# Mechanical homing
logger.debug('Homing X+Y mechanically')
printer.send_gcode('G28 X Y')
printer.send_gcode('M400')

# Move to Aruco location
logger.debug('Moving to Aruco marker')
printer.send_gcode('G1 X0 Y0 F6000')
printer.send_gcode('M400')
time.sleep(stabilize_delay)



2025-01-29 14:27:14,127 - machine                  - DEBUG   - Homing X+Y mechanically (1095807946.py:4)
2025-01-29 14:27:23,005 - machine                  - DEBUG   - Moving to Aruco marker (1095807946.py:9)


In [4]:
for i in range(5):
    logger.info(f'Running iteration {i}')

    # Take picture
    logger.debug('Capturing Image')
    image = camera.get_image()

    # Find markers
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #gray = cv2.rotate(gray, cv2.ROTATE_90_CLOCKWISE)
    corners, ids, rejected = detector.detectMarkers(gray)
    cv2.aruco.drawDetectedMarkers(image, corners, ids)

    if len(corners) != 1:
        raise RuntimeError('Calibration ArUco not found')


    location = np.flip(np.mean(corners[0][0], axis=0))
    center = np.array(image.shape[0:2]) // 2
    logger.debug(f'Found aruco at {location} (y, x) in image of size {image.shape} (y, x) and center {center} (y, x)')
    cv2.line(image, location[::-1].astype(int), center[::-1].astype(int), (255,0,0), 10)
    cv2.imwrite(f'{i}.png', image)

    delta = center - location
    logger.debug(f'Delta: {delta} (y, x), Distance: {np.sum(delta ** 2)}')

    scale_factor = 75
    movement = np.flip(np.round(delta / scale_factor, decimals=2))
    logger.debug(f'Using movement {movement} (x, y) (mm)')

    # Switch to relative, move
    printer.send_gcode('G91')
    printer.send_gcode(f'G1 X{-movement[0]} Y{movement[1]}')
    printer.send_gcode('G90')
    printer.send_gcode('M400')
    time.sleep(stabilize_delay)

2025-01-29 14:27:26,823 - machine                  - INFO    - Running iteration 0 (1897752381.py:2)
2025-01-29 14:27:26,824 - machine                  - DEBUG   - Capturing Image (1897752381.py:5)
2025-01-29 14:27:26,935 - machine                  - DEBUG   - Found aruco at [ 946.75 1841.75] (y, x) in image of size (2160, 4096, 3) (y, x) and center [1080 2048] (y, x) (1897752381.py:20)
2025-01-29 14:27:27,151 - machine                  - DEBUG   - Delta: [133.25 206.25] (y, x), Distance: 60294.625 (1897752381.py:25)
2025-01-29 14:27:27,152 - machine                  - DEBUG   - Using movement [2.75 1.78] (x, y) (mm) (1897752381.py:29)
2025-01-29 14:27:30,561 - machine                  - INFO    - Running iteration 1 (1897752381.py:2)
2025-01-29 14:27:30,562 - machine                  - DEBUG   - Capturing Image (1897752381.py:5)
2025-01-29 14:27:30,635 - machine                  - DEBUG   - Found aruco at [1074.5 2046. ] (y, x) in image of size (2160, 4096, 3) (y, x) and center [1080 

In [5]:
# Take picture
logger.debug('Capturing Image')
image = camera.get_image()

# Find markers
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
corners, ids, rejected = detector.detectMarkers(gray)
cv2.aruco.drawDetectedMarkers(image, corners, ids)

if len(corners) != 1:
    raise RuntimeError('Calibration ArUco not found')


location = np.flip(np.mean(corners[0][0], axis=0))
center = np.array(image.shape[0:2]) // 2
logger.debug(f'Found aruco at {location} (y, x) in image of size {image.shape} (y, x) and center {center} (y, x)')
cv2.line(image, location[::-1].astype(int), center[::-1].astype(int), (255,0,0), 10)
cv2.imwrite('2.png', image)

delta = location - center
logger.debug(f'Delta: {delta}, Distance: {np.sum(delta ** 2)}')


2025-01-29 14:27:44,820 - machine                  - DEBUG   - Capturing Image (2626297862.py:2)
2025-01-29 14:27:44,874 - machine                  - DEBUG   - Found aruco at [1080.25 2048.75] (y, x) in image of size (2160, 4096, 3) (y, x) and center [1080 2048] (y, x) (2626297862.py:16)
2025-01-29 14:27:45,101 - machine                  - DEBUG   - Delta: [0.25 0.75], Distance: 0.625 (2626297862.py:21)
